# PastePicture2Wallpaper

壁紙作成のための線画作成ツールです

以下のパラメータで値を弄ってください

- depth_threshold: 出力された深度情報をもとに線画化するための閾値
<!-- - image_upload_option: 画像をアップロードする方法 -->
  <!-- - `Upload from device`: デバイスからアップロード -->
  <!-- - `URL`: URLからダウンロード -->
  <!-- - `Mount Drive`: Google Driveのマウントを行う -->
<!-- - path_or_url: ダウンロード先のURLかマウント先のデバイスのパスを指定(Upload from devieの場合は無視されます) -->
- mode: 線画処理のモード
  - Anime : アニメ風の輪郭がはっきりとしている画像向け
- OS: 壁紙のOSの種類によって出力される線画の雰囲気が少し変わります
  - Mac風(灰色):パステルカラーの背景に合わせて半透明の灰色線画を出力
  - Mac風(白色):鮮やかな背景に半透明の白線画を出力
  - Windows風: 水色の背景にさらに明るい水色半透明の線画を出力
  - Ubuntu風: 紫-桃色の背景に馴染む桃色半透明の線画を出力

![](https://raw.githubusercontent.com/Ar-Ray-code/PastePicture2Wallpaper/main/images_for_readme/export-variation.png)

> デバイスからのアップロードのみ受け付けています
> 
> 直接URLを指定する場合は4番目のコードセルのURLを変更してコメントアウトを外し、以降のセルを実行（Ctrl+F10） を行ってください

In [49]:
depth_threshold = 150 #@param {type:"slider", min:0, max:255, step:1}

# image_upload_option = 'Upload from device' #@param ["Upload from device", "URL", "Mount Drive"] {allow-input: true}
# path_or_url = "/content/sample.jpg"#@param {type:"string"}

mode = 'Anime' #@param ["Anime"] {allow-input: true}
OS =  'Ubuntu' #@param ["Ubuntu","Windows","Mac-White","Mac-Gray"] {allow-input: true}

In [ ]:
# Upload from device
from google.colab import files
uploaded = files.upload()

#@markdown ### アップロードを (Choose Files)から行ってください
#@markdown Driveのマウントの場合はこのステップを飛ばして、新しくコードセルを作成して、`/content`に対象ファイルをコピーしてください

# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
#@markdown ###URLを指定する場合はこちらを編集し、upload.jpgで保存すること
# !rm sample.jpg
# !wget "https://drive.google.com/uc?export=download&id=1M1eDdinWr-aPT28cyOo01LlMFUnsBbRQ" -O upload.jpg

In [ ]:
!custom_path=basename path_or_url
!mv `basename path_or_url` /content/upload.jpg
!ffmpeg -i *.png upload.jpg
!ffmpeg -i *.jpg upload.jpg

In [ ]:
!pip install \
    tensorflow==2.8.0 \
    tensorflow-hub==0.12.0 \

In [8]:
import cv2
import numpy as np

import matplotlib.pyplot as plt
import tensorflow_hub as hub

module = hub.load("https://tfhub.dev/intel/midas/v2_1_small/1", tags=['serve'])
model = module.signatures['serving_default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
import tensorflow as tf

# 画像読み込み
image = cv2.imread('/content/upload.jpg')

# リサイズ
resize_image = cv2.resize(image, (256, 256))

# 正規化
resize_image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2RGB) / 255.0

# 形状変更
resize_image = resize_image.transpose(2, 0, 1)
resize_image = resize_image.reshape(1, 3, 256, 256)

# tensor形式へ変換
tensor = tf.convert_to_tensor(resize_image, dtype=tf.float32)

In [10]:
result = model(tensor)
predict_result = result['default'].numpy()
predict_result = np.squeeze(predict_result)

In [ ]:
# オリジナル画像のサイズにリサイズ
predict_result = cv2.resize(predict_result, (image.shape[1], image.shape[0]))

# 最大値が255になるよう変換
depth_max = predict_result.max()
data = ((predict_result / depth_max) * 255).astype(np.uint8)

# カラーマップ画像へ変換
color_map = cv2.applyColorMap(data, cv2.COLORMAP_TURBO)

# 表示
plt.imshow(color_map)

In [43]:
class image_utils:
    def median_filter(self, src, ksize):
        d = int((ksize-1)/2)
        h, w = src.shape[0], src.shape[1]

        dst = src.copy()

        for y in range(d, h - d):
            for x in range(d, w - d):
                dst[y][x] = np.median(src[y-d:y+d+1, x-d:x+d+1])
        return dst

    def export_edges_anime(self, image_raw: np.ndarray, depth_data: np.ndarray, threshold: int = 150):
        mask = depth_data > threshold
        # maskを二値化
        mask = mask.astype(np.uint8) * 255

        # imageとmaskの合成
        result = cv2.bitwise_and(image_raw, image_raw, mask=mask)
        result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        # メディアンフィルタ
        result = self.median_filter(result, ksize=5)
        # Canny輪郭検出
        edges = cv2.Canny(result, 199, 200)
        # MorphologyEx
        kernel = np.ones((5, 5), np.uint8)
        # edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
        edges = cv2.morphologyEx(edges, cv2.MORPH_GRADIENT, kernel)

        # 色の反転
        edges = cv2.bitwise_not(edges)
        edges = edges.astype(np.uint8) * 255 * 255
        edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

        return edges

    # 白を透明にして色付きを半透明の白にする
    def white2transparent(self, input_image: np.ndarray, os_type: str='Ubuntu'):
        if os_type == 'Ubuntu':
            image_png = cv2.cvtColor(input_image, cv2.COLOR_RGB2RGBA)
            for i in range(image_png.shape[0]):
                for j in range(image_png.shape[1]):
                    if image_png[i][j][0] == 255 and image_png[i][j][1] == 255 and image_png[i][j][2] == 255:
                        image_png[i][j][3] = 0
                    else:
                        image_png[i][j][0] = 255
                        image_png[i][j][1] = 200
                        image_png[i][j][2] = 255
                        image_png[i][j][3] = 200
        elif os_type == 'Windows':
            image_png = cv2.cvtColor(input_image, cv2.COLOR_RGB2RGBA)
            for i in range(image_png.shape[0]):
                for j in range(image_png.shape[1]):
                    if image_png[i][j][0] == 255 and image_png[i][j][1] == 255 and image_png[i][j][2] == 255:
                        image_png[i][j][3] = 0
                    else:
                        image_png[i][j][0] = 200
                        image_png[i][j][1] = 255
                        image_png[i][j][2] = 255
                        image_png[i][j][3] = 200
        elif os_type == 'Mac-White':
            image_png = cv2.cvtColor(input_image, cv2.COLOR_RGB2RGBA)
            for i in range(image_png.shape[0]):
                for j in range(image_png.shape[1]):
                    if image_png[i][j][0] == 255 and image_png[i][j][1] == 255 and image_png[i][j][2] == 255:
                        image_png[i][j][3] = 0
                    else:
                        image_png[i][j][0] = 200
                        image_png[i][j][1] = 200
                        image_png[i][j][2] = 200
                        image_png[i][j][3] = 200
        elif os_type == 'Mac-Gray':
            image_png = cv2.cvtColor(input_image, cv2.COLOR_RGB2RGBA)
            for i in range(image_png.shape[0]):
                for j in range(image_png.shape[1]):
                    if image_png[i][j][0] == 255 and image_png[i][j][1] == 255 and image_png[i][j][2] == 255:
                        image_png[i][j][3] = 0
                    else:
                        image_png[i][j][0] = 70
                        image_png[i][j][1] = 70
                        image_png[i][j][2] = 70
                        image_png[i][j][3] = 200
        else:
            print("cannot detect wallpaper OS")
        return image_png

In [13]:
image_utils_class = image_utils()

image = image_utils_class.export_edges_anime(image_raw=image, depth_data=data, threshold=depth_threshold)

#@markdown ## export_edges_anime関数
#@markdown ### 引数一覧
#@markdown - image_raw: 画像（読み込み時のまま）
#@markdown - depth_data: 深度データ(2次元のndarray・0~255)
#@markdown - threshold: 深度の閾値（0~255・int型）
#@markdown
#@markdown ### 出力一覧
#@markdown - image: マスク済み画像
#@markdown
#@markdown　---
#@markdown

In [50]:
image_utils_class = image_utils()
image_png = image_utils_class.white2transparent(image, OS)

#@markdown ## white2transparent関数
#@markdown 白領域を透明化する関数（色付きは半透明の白になります）

In [51]:
plt.imsave("/content/result.png",image_png)
from google.colab import files
files.download('result.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# Export depth
#@markdown convert.pyで深度情報を確認する時はこのセルを実行して`depth_result.png`をダウンロードします。

# # v----この行以下をコメントアウト--------v
# export_png = np.zeros((data.shape[0], data.shape[1], 3))
# export_png[:,:,0] = data/255
# export_png[:,:,1] = data/255
# export_png[:,:,2] = data/255

# plt.imsave("/content/depth_result.png",export_png)
# files.download('depth_result.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>